In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

import warnings
warnings.filterwarnings('ignore')

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import pandas as pd
import re

In [3]:
llm = AzureChatOpenAI(
    api_key=azure_openai_api_key,  # Azure OpenAI API Key
    api_version="2023-12-01-preview",  # API version (you can adjust it as needed)
    azure_endpoint=azure_openai_endpoint,  # Azure OpenAI endpoint
    deployment_name=deployment_name,  # Ensure deployment_name is the correct one you configured in Azure
    temperature=0  # Optional: Adjust the temperature for randomness in responses
)

C:\Users\antoa\AppData\Local\Temp\ipykernel_56288\1637258887.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


Ce processus est utile pour préparer des données textuelles structurées à partir d'un CSV en tant qu'objets structurés avec des métadonnées, ce qui est souvent nécessaire pour le traitement ultérieur ou l'analyse de texte.

In [4]:

# Charger le CSV
csv_path = "../../RGBD/table_produits/produits.csv"   # Remplace avec le chemin réel si besoin
df = pd.read_csv(csv_path, encoding='utf-8')

# Vérifier que la colonne "description" existe bien
if "description" not in df.columns:
    raise ValueError("La colonne 'description' n'existe pas dans le CSV ! Vérifie le nom des colonnes.")
# Vérification de la colonne "id_produit"
if "id_produit" not in df.columns:
    raise ValueError("La colonne 'id_produit' n'existe pas dans le CSV ! Vérifiez le nom des colonnes.")

# Transformer chaque ligne en objet Document avec métadonnées
documents = []
for index, row in df.iterrows():
    metadata = {"source": csv_path, "row": index,"id_produit": row["id_produit"]}

    if "saveur" in row:
        metadata["saveur"] = str(row["saveur"])  # Ajouter la saveur dans les métadonnées

    documents.append(Document(page_content=str(row["description"]), metadata=metadata))

print(f"Nombre de documents préparés : {len(documents)}")

Nombre de documents préparés : 1531


ce code te permet de transformer tes documents en embeddings vectoriels et de les sauvegarder dans un format FAISS, prêt à être utilisé pour des recherches ou des analyses futures sur tes données textuelles.

In [5]:
# Étape 3: Création et sauvegarde du vecteur FAISS avec métadonnées
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
    print("Vector store enregistré avec succès !")
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

C:\Users\antoa\AppData\Local\Temp\ipykernel_56288\4037170700.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



Vector store enregistré avec succès !


---------------------------------------------

https://www.youtube.com/watch?v=lyCLu53fb3g

Are the answers hight quality?
* Are they clear and undestandable?
* Are they correct? (relative to the knowledge base?)
* Are they dormatted in the desired manner?

What affects the quality?


automate evaluation

evaluation frameworks

In [12]:
query = "Quel produit contient des saveurs de fraise et bonbon ?"
# embedded_question = embeddings.embed_query(user_query)

In [13]:
# Fonction pour extraire les saveurs de la requête et les rechercher dans les résultats
def extract_flavors_from_query(query):
    """
    Cette fonction extrait les saveurs présentes dans la requête `query`
    et retourne une liste des saveurs trouvées parmi celles définies dans `flavor_list`.
    """
    # Liste des saveurs disponibles
    flavor_list = ["cassis", "lime", "fraise", "menthe", "mangue", "pêche", "bonbon"]

    found_flavors = []
    
    # Pour chaque saveur de la liste, vérifier si elle est mentionnée dans la requête
    for flavor in flavor_list:
        if re.search(rf'\b{flavor}\b', query, re.IGNORECASE):
            found_flavors.append(flavor)

    return found_flavors


# Fonction pour interroger le vecteur et récupérer les saveurs associées à la requête
def search_for_flavor(query):
    # Extraire les saveurs mentionnées dans la requête
    flavors_in_query = extract_flavors_from_query(query)

    # Afficher uniquement les saveurs extraites
    print(f"Saveurs extraites de la requête : {flavors_in_query}")

    # Retourner les saveurs extraites sous forme de chaîne, séparées par des virgules
    return ", ".join(flavors_in_query)

# Exemple d'appel à la fonction avec une requête utilisateur





In [14]:
flavors_in_query = search_for_flavor(query)

Saveurs extraites de la requête : ['fraise', 'bonbon']


In [15]:
retriever = vectorstore.as_retriever(
        search_type="similarity",  # Recherche des voisins les plus proches
        search_kwargs={
            "k": 10,  # Retourner les 10 voisins les plus proches
        }
    )

In [16]:
# Définition du prompt pour la génération de réponse
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""

# Initialisation du prompt
prompt = ChatPromptTemplate.from_template(prompt_template)

# Formatage des documents de contexte
def format_docs(flavors):
    # Création du contexte avec les saveurs extraites
    return f"Les saveurs extraites sont : {flavors}"

# Pipeline de question-réponse
qa_chain = (
    {
        "context": retriever | format_docs,  # Utilisation du retriever pour obtenir des contextes
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()  # Extraction de la réponse
)

# Exécution de la chaîne de question-réponse et affichage du résultat
# Ici, tu passes les saveurs extraites en tant que contexte dans le modèle
response = qa_chain.invoke(query, filter={"saveur": flavors_in_query})
# Affichage du résultat
print(f"Réponse générée par le modèle : {response}")

Réponse générée par le modèle : Le produit qui contient des saveurs de fraise et bonbon est le "Fraise Candy", un e-liquide fabriqué par Prestige Fruits pour la gamme Sweety Fruits.


In [17]:
source = retriever.invoke(query, filter={"saveur": flavors_in_query})

Relevance Score  (similarité cosinus) avec HuggingFaceEmbeddings
 La similarité entre les trois éléments (query, réponse, et source) est mesurée ici. Vous utilisez HuggingFaceEmbeddings pour encoder vos phrases et calculer la similarité cosinus entre la question et la réponse ainsi qu'entre la réponse et le contexte.
 HuggingFaceEmbeddings encode chaque phrase (query, response, source) en vecteurs d'embeddings.
Ensuite, vous utilisez la similarité cosinus pour comparer les vecteurs entre eux et obtenir une mesure de proximité entre les textes.
Cela vous permet de mesurer à quel point la réponse est pertinente par rapport à la question et à quel point la réponse est en accord avec le contexte donné.

In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Charger le modèle SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Fonction pour calculer la similarité cosinus entre query, response et source
def calculate_sentence_similarity(query, response, source):
    # Assurez-vous que les entrées sont des chaînes de caractères
    texts = [str(query), str(response), str(source)]
    
    # Obtenir les embeddings
    embeddings = model.encode(texts)
    
    # Calculer la similarité cosinus entre les embeddings
    similarity_query_response = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    similarity_response_source = cosine_similarity([embeddings[1]], [embeddings[2]])[0][0]
    
    return similarity_query_response, similarity_response_source

similarity_query_response, similarity_response_source = calculate_sentence_similarity(query, response, source)
print(f"Similarité entre la question et la réponse : {similarity_query_response}")
print(f"Similarité entre la réponse et le contexte : {similarity_response_source}")


Similarité entre la question et la réponse : 0.7342333197593689
Similarité entre la réponse et le contexte : 0.7779853343963623


In [19]:
# Créer un objet HuggingFaceEmbeddings avec le modèle souhaité
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)

# Fonction pour calculer la similarité cosinus entre la question, la réponse et le contexte
def calculate_sentence_similarity(query, response, source):
    # Encoder les phrases (query, response, source) en embeddings
    embeddings = embedding.embed_documents([query, response, source])
    
    # Calculer la similarité cosinus entre les embeddings
    similarity_query_response = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    similarity_response_source = cosine_similarity([embeddings[1]], [embeddings[2]])[0][0]
    
    return similarity_query_response, similarity_response_source

# Afficher les résultats
print(f"Similarité entre la question et la réponse : {similarity_query_response}")
print(f"Similarité entre la réponse et le contexte : {similarity_response_source}")


Similarité entre la question et la réponse : 0.7342333197593689
Similarité entre la réponse et le contexte : 0.7779853343963623


D
Le groundedness score mesure la pertinence de la réponse par rapport au contexte, mais avec un accent sur la fidélité de la réponse par rapport à l'information fournie dans le contexte (c'est-à-dire, si la réponse est bien fondée et cohérente avec les informations sources).
SentenceTransformer est utilisé ici pour obtenir les embeddings des phrases.
Vous calculez la similarité cosinus entre la réponse et le contexte pour obtenir une mesure de la "cohérence" de la réponse par rapport au contexte.
Cela mesure à quel point la réponse est en accord avec les informations présentes dans le contexte, sans nécessairement prendre en compte la question elle-même.
Utilité : Ce score est utile pour évaluer si une réponse est bien ancrée (grounded) dans le contexte donné, c'est-à-dire si elle est en cohérence avec les informations contextuelles.

In [20]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Charger le modèle SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Fonction pour calculer la similarité cosinus entre la réponse et le contexte
def calculate_groundedness_score(response, source):
    # S'assurer que response et source sont bien des chaînes de caractères
    response_text = str(response)
    source_text = str(source)
    
    # Encoder les phrases (réponse et contexte) en embeddings
    embeddings = model.encode([response_text, source_text])
    
    # Calculer la similarité cosinus entre la réponse et le contexte
    groundedness_score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    
    return groundedness_score


# Calculer le groundedness score
groundedness_score = calculate_groundedness_score(response, source)

# Afficher le résultat
print(f"Groundedness Score (réponse et contexte) : {groundedness_score}")


Groundedness Score (réponse et contexte) : 0.7779853343963623


Résumé des différences :
Relevance Score :

Mesure la pertinence de la réponse vis-à-vis de la question et du contexte.
Utilise HuggingFaceEmbeddings pour encoder et calculer la similarité cosinus entre trois éléments (question, réponse, contexte).
Permet de voir si la réponse est pertinente pour la question et en accord avec le contexte.
Groundedness Score :

Mesure la cohérence ou l'alignement de la réponse avec le contexte donné.
Utilise SentenceTransformer pour encoder et calculer la similarité cosinus entre la réponse et le contexte.
Permet d'évaluer si la réponse est bien "fondée" sur le contexte.

bulk eval

In [34]:
queries = [
    "Je veux un e-liquide avec cassis et lime",
    "Quel produit avec fraise",
    "Je veux saveur frais"
    ]

In [35]:
# Listes pour stocker les réponses et les sources
responses = []
sources = []

In [36]:
# Exécution de la chaîne pour chaque query
for query in queries:
    # Effectuer la recherche du contexte avec le retriever
    source = retriever.invoke(query) #, filter={"saveur": flavors_in_query})
    
    # Exécuter le pipeline pour obtenir la réponse
    response = qa_chain.invoke(query, filter={"saveur": flavors_in_query})
    
    # Ajouter la réponse et la source aux listes respectives
    responses.append(response)
    sources.append(source)

In [37]:
for i, query in enumerate(queries):
    print(f"Query: {query}")
    print(f"Réponse: {responses[i]}")
    print(f"Source: {sources[i]}")
    print("-" * 40)

Query: Je veux un e-liquide avec cassis et lime
Réponse: Il existe plusieurs e-liquides avec des saveurs de cassis et de lime. Voici quelques options :

1. **Cassis Noir** de la collection Petit Nuage par Roykin, qui propose des saveurs de cassis et de lime.

2. **Cassis Lime** de la collection Freeze par Liquideo, qui inclut également des saveurs de baie et de baie noire.

3. **Melon Cassis Banane** de la collection Fizz and Freeze par Liquideo, qui inclut des saveurs de cassis, lime, melon, et banane.

4. **Cassigre** de la collection Amalgam par e.Tasty, qui propose des saveurs de grenade, cassis, et lime.

Ces options vous offrent une variété de mélanges avec cassis et lime, parfois accompagnés d'autres saveurs.
Source: [Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 723, 'id_produit': 725, 'saveur': 'cassis, lime'}, page_content="Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de

In [38]:
sources

[[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 723, 'id_produit': 725, 'saveur': 'cassis, lime'}, page_content="Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux